In [169]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import math
import csv
import numpy as np
import pickle

import gensim

#either use google's word2vec or use your own trained model and check which one is working better for you.

#Choose from one of the below models

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# model = Word2Vec.load('word2vec_model.pkl')

with open('processed_comments', 'rb') as f:
     clean_comments = pickle.load(f)
        
with open('unprocessed_comments', 'rb') as f:
     unclean_comments = pickle.load(f)


In [194]:
len(clean_comments)

In [171]:
#Tfidf scores computation

n = len(clean_comments)
start_index = len(clean_comments)-n
clean_comments = clean_comments[start_index:len(clean_comments)]
unclean_comments = unclean_comments[start_index:len(unclean_comments)]
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf  = TfidfVectorizer(use_idf=True)
tfidf_data = []
for i in range(len(clean_comments)):
    temp_vec = []
    for j in range(len(clean_comments[i])):
        temp_vec.append(str(clean_comments[i][j]))
    tfidf_data.append(' '.join(temp_vec))
    
tfidf_scores = tfidf.fit_transform(tfidf_data)
tfidf.get_feature_names()
vocab = tfidf.vocabulary_


In [195]:
# Top n words from text
# This cell is optional to use as this induces biasing for the clusters. Use this if you know your domain specific 
# keywords and want them to be clearly separated from the rest of the clusters. Run this but assign the weights to the
# first if in the next cell 

representational_vectors = []
words = []

#change this as per your domain and customer pain points. These keywords are just assumptions.
words_vec = ['resolution', 'case', 'dispute', 'claim', 'receive', 'seller', 'buyer', 'refund', 'account', 'send', 'item', 'email', 'resolve', 'contact', 
        'close', 'open', 'transaction', 'message', 'claim', 'issue', 'problem', 'payment', 'phone', 'ship', 'find', 'view', 'credit', 'card', 
        'track', 'ebay', 'bank', 'center', 'show', 'report', 'address', 'hold', 'limitation', 'file', 'answer', 'deliver', 'website', 'email', 'respond', 'process', 
        'complaint', 'review', 'click', 'tab', 'navigation', 'spin', 'reload', 'image', 'evidence', 'scam', 'fraud', 'allow', 
        'unable', 'info', 'header', 'scroll', 'page', 'link', 'system', 'reason', 'access', 'see', 'unauthorized', 'cancel', 'chargeback', 'fix', 'proof', 'confirm', 'online', 'experience', 
        'pending', 'dashboard', 'restore', 'status', 'describe', 'confuse', 'appear', 'feedback', 'upload', 'load', 'refresh', 'communication', 
        'gmail', 'button', 'text', 'protection', 'steal', 'debit', 'record', 'enter', 'input', 'submit', 'invoice', 'password', 'security', 'statement', 'withdraw', 
        'fault', 'touch', 'app', 'browser', 'chrome', 'android', 'mobile', 'impossible', 'computer', 'screen', 'form', 'automate', 'delete', 'remove', 'advise', 
        'notification', 'multiple', 'raise', 'chat', 'report', 'create', 'easier', 'human', 'appeal', 'circle', 'visa', 'policy','reopen', 'attach', 'quick', 'select', 'method', 'subscription', 
        'automatic', 'authorize', 'documentation', 'prompt', 'query', 'decline', 'progress', 'estimate', 'download', 'exchange', 'service', 'faster', 'freeze', 
         'connect', 'error', 'wrong', 'permission', 'escalate', 'disappear', 'inconvenience', 'chargeback', 'duplicate', 'alert', 'initiate', 'design', 'wallet', 'directions' ]

for i in range(len(words_vec)):
    words.append(words_vec[i])

final_words_dict = {}
final_words = []
count = 0
for word in words:
    if ((str(word) in final_words_dict) == False):
        final_words_dict[str(word)] = count
        count = count + 1
        final_words.append(str(word))
        word2vec_words = model.wv.most_similar (str(word),topn=5)
        for i in range(len(word2vec_words)):
            item = word2vec_words[i][0]
            if (item in final_words_dict)== False:
                final_words.append(item)
                final_words_dict[item] = count
                count = count + 1
                

In [196]:
#comment_vectors simplified

def rms(sentence_vector):
    square_sum = 0;
    for i in range(len(sentence_vector)):
        square_sum = square_sum + (sentence_vector[i]*sentence_vector[i])
    rms_value = math.sqrt(square_sum/len(sentence_vector))
    rms_vector = sentence_vector
    for i in range(len(sentence_vector)):
        rms_vector[i] = sentence_vector[i]/rms_value
    return rms_vector

comment_vectors = []
total_empty = 0
for i in range(len(clean_comments)):
    count = 0
    word_vector = model.wv[clean_comments[0][0]] - model.wv[clean_comments[0][0]]
    for j in range(len(clean_comments[i])):
        if((str(clean_comments[i][j]) in final_words_dict) == True ):
            count = count + 1
            #Assign weight i.e multiply weight by a factor (10-20) if you used above cell
            weight = model.wv[str(clean_comments[i][j])]*tfidf_scores[(i,vocab[clean_comments[i][j]])] * 30
            word_vector = word_vector + weight
        elif((str(clean_comments[i][j]) in model.wv.vocab) == True):
            weight = model.wv[str(clean_comments[i][j])]*tfidf_scores[(i,vocab[clean_comments[i][j]])]
            word_vector = word_vector + weight
        else:
            weight = tfidf_scores[(i,vocab[clean_comments[i][j]])]
            word_vector = word_vector + weight
            
    rms_vector = rms(word_vector)
    comment_vectors.append(rms_vector)
    if(count == 0):
        total_empty = total_empty + 1
        
print total_empty

In [197]:
len(comment_vectors)

In [175]:
#Visualizing the comments in 2-D plane using TSNE Algorithm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
from sklearn.manifold import TSNE

RS = 20150101
vectors_tsne = TSNE(random_state=RS).fit_transform(comment_vectors)

In [198]:
f = plt.figure(figsize=(20, 20))
ax = plt.subplot(aspect='equal')
sc = ax.scatter(vectors_tsne[:,0], vectors_tsne[:,1], lw=0, s=40)
ax.axis('tight')

In [177]:
with open('comments_vectors', 'wb') as f:
    pickle.dump(comment_vectors, f)

with open('vectors_tsne', 'wb') as f:
    pickle.dump(vectors_tsne, f)

In [178]:
with open('comments_vectors', 'rb') as f:
     comment_vectors = pickle.load(f)

In [199]:
# computing statistics for feedbacks 

rating_data = pd.read_csv('Landing_page_feedbacks.csv')
rating_data = rating_data['Overall Rating']
token_count = np.array([ len(comment) for comment in clean_comments])
token_count = token_count[-25660:]
total_rating = 0
for i in range(len(rating_data)):
    total_rating = total_rating + float(rating_data[i])

print "Avg words per feedback:", np.mean(token_count)
print "Std Deviation of feedbacks:", np.std(token_count)
print "Length of max feedback:", np.max(token_count)
print "Length of min feedback:", np.min(token_count)
print "Avg Rating for 2017-2018:", total_rating/len(rating_data)

In [57]:
# Checking which number of clusters to choose

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
range_n_clusters = [5, 7, 9, 10, 15, 20, 25, 30, 35, 40, 50, 60]
train_vectors = comment_vectors

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(train_vectors)
    print(cluster_labels)
    silhouette_avg = silhouette_score(train_vectors, cluster_labels, metric="cosine")
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

In [200]:
#Choose your cluster number based on silhouette score and the plotted data points.

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

# Set number of clusters you want
n_clusters = 40
train_vectors = comment_vectors

clusterer = KMeans(n_clusters=n_clusters, random_state=35)
cluster_labels = clusterer.fit_predict(train_vectors)
silhouette_avg = silhouette_score(train_vectors, cluster_labels, metric='cosine')
print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)

In [181]:
clusters = []
clean_clusters = []
actual_comments = unclean_comments
actual_clean_comments = clean_comments
for i in range(n_clusters):
    clusters.append([])
    clean_clusters.append([])

for i in range(len(cluster_labels)):
    label = cluster_labels[i];
    clusters[label].append(actual_comments[i])
    clean_clusters[label].append(actual_clean_comments[i])


In [201]:
f = plt.figure(figsize=(20, 20))
ax = plt.subplot(aspect='equal')
sc = ax.scatter(vectors_tsne[:,0], vectors_tsne[:,1], lw=0, s=40, c=cluster_labels, cmap = 'prism')

ax.axis('tight')

In [72]:
with open('comments_vectors', 'wb') as f:
    pickle.dump(comment_vectors, f)

In [183]:
# Summarization of Clusters.

from summa import summarizer

data_for_summary = []
    
# print(' '.join(clean_clusters[5][0]))
for i in range(len(clean_clusters)):
    temp_data = []
    for j in range(len(clean_clusters[i])):
        text = ' '.join((clean_clusters[i][j]))
        temp_data.append(text)
    data_for_summary.append(temp_data)
    
for i in range(len(clean_clusters)):
    temp_string =''
    for j in range(len(data_for_summary[i])):
        temp_string = temp_string + str(data_for_summary[i][j]) + '. '
    data_for_summary[i] = temp_string

#Set number of words you want in the summary of the cluster
words_in_summary = 100
summary_clusters = []
for i in range(len(data_for_summary)):
    summary_clusters.append(summarizer.summarize(data_for_summary[i], words = words_in_summary))




In [202]:
# Pretty printing of clusters with summaries and count of feedbacks in cluster and the avg rating for that cluster

temp = np.zeros((len(clean_clusters),2)).astype(int)
for i in range(len(clean_clusters)):
    temp[i][0] = len(clean_clusters[i])
    temp[i][1] = i

temp = temp[np.argsort(-temp[:, 0])]

def compute_avg_rating(cluster_number):
    total = 0;
    for i in range(len(clusters[cluster_number])):
        total = total + float((clusters[cluster_number])[i][1])
    return total/len(clusters[cluster_number])

for i in range(len(temp)):
    print "Cluster", temp[i][1],":", temp[i][0], "  Avg. Rating:", float("{0:.2f}".format(compute_avg_rating(temp[i][1])))
    print(summary_clusters[temp[i][1]])
    print('\n')



In [62]:
# Seeing the full content of the clusters. Cluster numbers start from 0. These clusters are printed
# in reverse chronological order

# Set cluster_num to view it
cluster_num = 23
clusters[cluster_num][::-1]

In [203]:
# This script searches for the specific keywords in the whole corpus. You can provide the keywords you want to find.
# Returns the feedbacks having all the keywords.

import string

def fetch_caseid_feedbacks(search):
    resultant_cluster = []
    keys = search.split()
    keys_dict = {}
    for i in range(len(keys)):
        if( keys[i] not in keys_dict):
            keys_dict[keys[i]] = i
        
    for i in range(len(unclean_comments)):
        feedback_tokens = unclean_comments[i][0].split()
        count = 0
        comment_dict = {}
        for j in range(len(feedback_tokens)):
            if(str(feedback_tokens[j]) not in comment_dict):
                    comment_dict[feedback_tokens[j]] = j
        for key in keys_dict:
            if key not in comment_dict:
                count = count + 1
                break
        if count == 0:
            resultant_cluster.append(unclean_comments[i])
    return resultant_cluster

keywords =  'your keywords separated by spaces'
fetch_caseid_feedbacks(keywords)

In [185]:
# Function to get the top n topics from clusters 

def cluster_topics(clean_clusters, len_of_topics, topics_per_cluster, words_in_each_topic):
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.decomposition import LatentDirichletAllocation
    
    topics_for_all = []
    for i in range(len(clean_clusters)):
        cluster_data = clean_clusters[i]
        content = []
        for i in range(len(cluster_data)):
            content.append(str(' '.join(cluster_data[i])))

        vectorizer = CountVectorizer(ngram_range = (len_of_topics, len_of_topics))
        vect = vectorizer.fit_transform(content)
        lda = LatentDirichletAllocation(n_components = topics_per_cluster)
        topics = lda.fit_transform(vect)
        sorting = np.argsort(lda.components_)[:,::-1]
        features = np.array(vectorizer.get_feature_names())
        
        topics_for_all.append([topics_per_cluster,features,
                               sorting, 1, words_in_each_topic])
    return topics_for_all


In [186]:

# Arguments for cluster_topics ---> First = clean_clusters, second = len_of_topics, third = num of topics per cluster
topics_for_all = cluster_topics(clean_clusters, 3, 3, 3)
def pretty_printing_topics(topics_for_all, cluster_number):
    import mglearn
    mglearn.tools.print_topics(topics=range(topics_for_all[cluster_number][0]), feature_names=topics_for_all[cluster_number][1],
                                  sorting=topics_for_all[cluster_number][2], topics_per_chunk = topics_for_all[cluster_number][3], n_words = topics_for_all[cluster_number][4])
    


In [208]:
# Cluster for which you want the top n topics

clust_num = 11
pretty_printing_topics(topics_for_all, clust_num)